-----
# cartoGRAPHs - Figure 2C
-----

In [2]:
from cartoGRAPHs import *

2021-11-23 11:11:54.361647: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-23 11:11:54.362021: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


_____
# HUMAN PPI NETWORK
_____

In [3]:
organism = 'human'
G = load_graph(organism)

d_centralities = load_centralities(G, organism)
df_centralities = pd.DataFrame(d_centralities).T
df_centralities.columns = ['degree','closeness', 'betweeness', 'eigenvector']
essential_genes,non_ess_genes,notdefined_genes = load_essentiality(G, organism)

d_gene_sym = load_genesymbols(G, organism)
l_features = list(d_gene_sym.values())

# Human specific > to include Disease Ontology
if organism == 'human':
    d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
    d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
    d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
    d_names_do = {y:x for x,y in d_do_names.items()}
else: 
    pass

netlayout = 'global'
DM = load_datamatrix(G,organism, netlayout)

_____ 
## SPATIAL EMBEDDING
_____

### 2D PORTRAIT

In [290]:
%%time 

posG_umap2D = layout_portrait_umap(G,DM,2,n_neighbors=8, spread=1, min_dist=0.0, metric='cosine')  #layout_portrait_umap(G,DM,2,n_neighbors=8, spread=6, min_dist=0.9, metric='cosine') 
posG = posG_umap2D

CPU times: user 16.9 s, sys: 652 ms, total: 17.6 s
Wall time: 14.4 s


____________
## VISUAL SETTINGS + PLOT PREPARATION
____________

### NODES GENERAL 

In [280]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 0.1

scale_factor = 0.55
size = list(draw_node_degree(G, scale_factor).values())

scale_factor3D = 0.025
size3d = list(draw_node_degree_3D(G, scale_factor3D).values())

nodesglow_diameter = 8.0
nodesglow_transparency = 0.01 # 0.01

opacity_edges = 0.05
width = 0.1

---
## TOPOGRAPHIC MAP 
---

In [58]:
z_feat = 'disease'
DM_feature  = pd.read_pickle('input/Features_Disease_Dataframe_Human.pickle')

# set gene list (= G.nodes())
genes_assoc = []
for i in DM_feature.index:
    genes_assoc.append(str(i))

genes_notassoc = [] 
for g in G.nodes():
    if g not in genes_assoc:
        genes_notassoc.append(g)

features_counted = (DM_feature == 1).astype(int).sum(axis=1)
d_features_counted = dict(zip(genes_assoc, list(features_counted)))
    
d_rest = {}
for i in genes_notassoc: 
    d_rest[i] = -20
    
d_param_unsorted = {**d_features_counted, **d_rest}
d_z = {key:d_param_unsorted[key] for key in G.nodes()}
    
d_clos = {}
for k,v in d_centralities.items():
    d_clos[k] = v[1]

d_nodecolors = d_clos 
col_pal = 'YlOrRd'

d_colors = color_nodes_from_dict(G, d_nodecolors, palette = col_pal)
colors = list(d_colors.values())
edge_color = '#ACACAC'
width_edges = 0.15
opacity_edges = 0.1

umap_nodes = get_trace_nodes_3D(posG, l_features, colors, size, opacity_nodes)
umap_edges = get_trace_edges_3D(G, posG, edge_color, opac = 0.05, linewidth = 0.5)
data = [umap_edges, umap_nodes]

---
## PLOT 
---

In [59]:
# ---------------------------------------------------------------------------
#
# Choose a 2D layout as basis
#
# 1. either calculate one e.g.: 
# posG_2Dportrait = layout_portrait_umap(G,DM,2,n_neighbors=20, spread=1, min_dist=0.0, metric='cosine') 
# ---------------------------------------------------------------------------
#
# 2. or use a precalculated 2D layout 
posG_2D = posG_umap2D
# ---------------------------------------------------------------------------

posG = layout_topographic(posG_2D, d_z)

path = 'output_plots/Topographic/'
schema = 'light' #'dark'
fname = 'topographic_z_'+z_feat+'_'+netlayout+'_'+organism+'_'+schema

plot_3D(data,path,fname, schema)

'output_plots/Topographic/topographic_z_disease_local_human_light.html'